In [1]:
using Parameters       # @unpack
using QuantEcon        # rouwenhorst
using LinearAlgebra    # dot
using Roots            # findzero
using SparseArrays     # SparseMatrixCSC
using IterativeSolvers # powm!
using Plots            # Plots
using NBInclude        # including notebooks instead of jl files
using MAT              ####  write the .m file 
using DataFrames
using Printf



In [2]:
@nbinclude("Structures_Reiter_Comp.ipynb");
@nbinclude("Utils_Reiter_Comp.ipynb");
@nbinclude("SolveAiyagari_Reiter_Comp.ipynb");

In [3]:
params = Params(
    Rt=1.0, #### Will be determined 
    β=0.99,
    α=0.36,
    δ=0.025,
    γ=1.,
    ϕ=0.5,
    χ = 1.0,
    ϵ = 6.0,
    κ= 100.0, #### price adjustment not taxe
    tl=0.00, #### 0.28
    tk=0.00, #tk=0.36
    Tt=0.00, #0.104
    na=100,
    a_min=1e-9, 
    a_max=1000.0,
    curv_a=4.0,
    ny=5,
    ρy= 0.99, #0.9615 #0.99
    σy= 0.123#0.0835#0.123 #0.14059684585185987 #0.835
    );

# R_opti = find_zero(find_KSL, (1.004,1.005)) # Uncomment if params is changed 
R_opti =  1.0044102383608413


1.0044102383608413

In [9]:

####################################
## STEADY-STATE 3 PROJECTIONS
####################################

params_opti = Params(
    Rt=R_opti,
    β=0.99,
    α=0.36,
    δ=0.025,
    γ=1.,
    ϕ=0.5,
    χ = 1.0,
    ϵ = 6.0,
    κ= 100.0, #### price adjustment not taxe
    tl=0.00, #### 0.28
    tk=0.00, #tk=0.36
    Tt=0.00, #0.104
    na=100,
    a_min=1e-9, 
    a_max=1000.0,
    curv_a=4.0,
    ny=5,
    ρy= 0.99, #0.9615 #0.99
    σy= 0.123#0.0835#0.123 #0.14059684585185987 #0.835
    );
        
solution_opti = steady(params_opti);
describe_solution(solution_opti,params_opti)


Solved in 800 iterations


Dict{String, Float64} with 11 entries:
  "total tax/Y"                 => 0.0
  "Share of constrained agents" => 0.194226
  "L"                           => 0.361034
  "MPC"                         => 0.146496
  "I/Y"                         => 0.306016
  "C/Y"                         => 0.693984
  "Gini"                        => 0.734675
  "G/Y"                         => 2.70567e-15
  "B/Y"                         => -2.0984e-13
  "Transfers/Y"                 => 0.0
  "K/Y"                         => 12.2406

# Refined Truncation

In [18]:
## Truncation Refined

@nbinclude("Projection_refined_Reiter_Comp.ipynb");
Vidio = repeat([20],params.ny)

N=5;
proj_refined = refinedProject_plan(N,Vidio,solution_opti,params_opti,new_ξs=false)
Write_Dynare(proj_refined,solution_opti,params_opti,"Refined")

In [16]:
ρy= 0.99 
σy= 0.123


mc    = rouwenhorst(params_opti.ny, ρy, σy)
Trans = collect(mc.p')
Trans[findall(x->x<=5*10^-5,Trans)] .= zero(Trans[1,1])
for i = 1:params_opti.ny
    Trans[i,i] += one(Trans[1,1]) - sum(Trans,dims=1)[i]
end

Transv = vec(Trans)


25-element Vector{Float64}:
 0.98014999875
 0.01970149750000002
 0.00014850375000000028
 0.0
 0.0
 0.004925374375000005
 0.9802238768749999
 0.014776496250000014
 7.425250000000014e-5
 0.0
 ⋮
 7.425250000000014e-5
 0.014776496250000014
 0.980223876875
 0.004925374375000005
 0.0
 0.0
 0.00014850375000000028
 0.01970149750000002
 0.98014999875

In [17]:

@nbinclude("Projection_Reiter_Comp.ipynb")
Vind,Wind,resE,apol,cpol,lpol = Projection_Reiter(params_opti,solution_opti)
ytype = kron(collect(StepRange(1, Int8(1), params_opti.ny)),ones(Int64,params_opti.na)); # on the initial grid


file = matopen("../dynamics/todynare_Comp_Reiter.mat", "w")

write(file, "alpha", params_opti.α)
write(file, "beta", params_opti.β)
write(file, "delta", params_opti.δ)
write(file, "gamma", params_opti.γ)
write(file, "chi", params_opti.χ)
write(file, "phi", params_opti.ϕ)      
write(file, "Trans", reshape(Transv,params_opti.ny,params_opti.ny))
write(file, "abar", minimum(params_opti.aGrid))
write(file, "ys", params_opti.ys)
write(file, "Vind", Vind)
write(file, "Wind", Wind)
write(file, "resE", resE)
write(file, "na", params_opti.na)
write(file, "ny", params_opti.ny)
write(file, "ytype", ytype)
write(file, "apol", apol') # policy rule ap(a) ie Va'
write(file, "aGrid", repeat(params_opti.aGrid,params_opti.ny*2))
write(file, "D_ss", vec(solution_opti.stationaryDist))
write(file, "Cpol",cpol')
write(file, "Lpol",lpol')
write(file, "polA_ss",vec(solution_opti.ga))
write(file, "w",solution_opti.w)
write(file, "R",solution_opti.R)
write(file, "A",solution_opti.A)
write(file, "K",solution_opti.K)
write(file, "Ltot",solution_opti.L)
write(file, "Ctot",solution_opti.C)
write(file, "B",solution_opti.B)
write(file, "Resid",solution_opti.residEuler)
close(file)
